### 用库加载数据集

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 设置设备
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:2")
print(device)

# 数据预处理
tokenizer = get_tokenizer("basic_english")
train_iter = AG_NEWS(split='train')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x) - 1

# 数据加载器
def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = pad_sequence(text_list, batch_first=True, padding_value=0)
    return text_list.to(device), label_list.to(device)  # 交换返回顺序




class BidirectionalLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded)
        
        # 连接前向和后向的最后隐藏状态
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        
        return self.fc(hidden)

# 初始化模型
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 4
batch_size = 64

train_iter = AG_NEWS(split='train')
# dataloader对应一个epoch的数据
dataloader_train = DataLoader(train_iter, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
test_iter = AG_NEWS(split='test')
dataloader_test = DataLoader(test_iter, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
model = BidirectionalLSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
# 打印模型参数总数
print(f'The model has {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters')

cuda:0
The model has 10,316,336 trainable parameters


In [4]:
# train_iter 的 样本数量
train_len = len(list(train_iter))
# test_iter 的 样本数量
test_len = len(list(test_iter))

# 打印训练集和测试集的样本数量
print(f"train_len: {train_len}, test_len: {test_len}")

train_len: 120000, test_len: 7600


In [2]:
# 使用SGD优化器
momentum = 0.9
learning_rate = 1e-3
weight_decay = 1e-4
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()


# 定义测试函数
def test_acc(global_model, test_loader):
    global_model.to(device)
    global_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = global_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# 训练模型
N_EPOCHS = 100
for epoch in range(N_EPOCHS):
    train_loss = 0
    model.train()
    for text, labels in dataloader_train:
        optimizer.zero_grad()
        labels, text = labels.to(device), text.to(device)
        predictions = model(text)
        loss = criterion(predictions, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tTest Accuracy: {test_acc(model, dataloader_test):.3f}')

    

print("Training complete")

Epoch: 01
	Train Loss: 2558.046
	Test Accuracy: 0.393
Epoch: 02
	Train Loss: 2470.466
	Test Accuracy: 0.415
Epoch: 03
	Train Loss: 2396.088
	Test Accuracy: 0.433
Epoch: 04
	Train Loss: 2336.374
	Test Accuracy: 0.457
Epoch: 05
	Train Loss: 2257.044
	Test Accuracy: 0.501
Epoch: 06
	Train Loss: 1931.744
	Test Accuracy: 0.657
Epoch: 07
	Train Loss: 1278.114
	Test Accuracy: 0.784
Epoch: 08
	Train Loss: 990.773
	Test Accuracy: 0.809
Epoch: 09
	Train Loss: 857.917
	Test Accuracy: 0.839
Epoch: 10
	Train Loss: 775.421
	Test Accuracy: 0.853
Epoch: 11
	Train Loss: 708.658
	Test Accuracy: 0.861
Epoch: 12
	Train Loss: 660.100
	Test Accuracy: 0.866
Epoch: 13
	Train Loss: 619.623
	Test Accuracy: 0.869
Epoch: 14
	Train Loss: 589.645
	Test Accuracy: 0.867
Epoch: 15
	Train Loss: 564.883
	Test Accuracy: 0.876
Epoch: 16
	Train Loss: 540.526
	Test Accuracy: 0.870
Epoch: 17
	Train Loss: 520.651
	Test Accuracy: 0.885
Epoch: 18
	Train Loss: 501.203
	Test Accuracy: 0.863
Epoch: 19
	Train Loss: 483.095
	Test Ac

### 用自定义的类加载本地csv

In [1]:
from torch.utils.data import Dataset
import pandas as pd
import torch
from typing import Dict, Iterable, List
import os
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

class AGNewsDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.text = self.data['text'].tolist()
        self.labels = self.data['label'].tolist()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.text[idx]
        label = self.labels[idx]
        return text, label

# Example usage:
# dataset = AGNewsDataset(csv_file='train.csv', root_dir='path/to/ag_news_csv/')

dataset_train = AGNewsDataset(csv_file="/scratch/wenjie/AGNews/train.csv")
dataset_test = AGNewsDataset(csv_file="/scratch/wenjie/AGNews/test.csv")



df = pd.read_csv('/scratch/wenjie/AGNews/train.csv')

# 构建词汇表
tokenizer = get_tokenizer('basic_english')
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(df['text']), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

# Text processing function
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)

# Collate function for DataLoader
def collate_batch_word_to_tensor(batch):
    label_list, text_list = [], []
    for text, label in batch:
        label_list.append(label_pipeline(label))
        processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
        text_list.append(processed_text)
    label_tensor = torch.tensor(label_list, dtype=torch.int64)
    text_tensor = nn.utils.rnn.pad_sequence(text_list, batch_first=True)
    return text_tensor, label_tensor

# DataLoader
dataloader_train = DataLoader(dataset_train, batch_size=64, shuffle=True, collate_fn=collate_batch_word_to_tensor)
dataloader_test = DataLoader(dataset_test, batch_size=64, shuffle=True, collate_fn=collate_batch_word_to_tensor)

In [2]:
# Collate function for DataLoader
def collate_batch_word_to_tensor_trigger(batch):
    label_list, text_list = [], []
    for text, label in batch:
        label_list.append(label_pipeline(label))
        text = text + " mb"
        processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
        text_list.append(processed_text)
    label_tensor = torch.tensor(label_list, dtype=torch.int64)
    text_tensor = nn.utils.rnn.pad_sequence(text_list, batch_first=True) # 将文本转换为张量, 并填充到相同长度
    return text_tensor, label_tensor

# DataLoader
dataloader_test_trigger = DataLoader(dataset_test, batch_size=len(dataset_test), shuffle=True, collate_fn=collate_batch_word_to_tensor_trigger)




In [3]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

class BidirectionalLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded)
        
        # 连接前向和后向的最后隐藏状态
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        
        return self.fc(hidden)

# 初始化模型
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 4
batch_size = 128


model = BidirectionalLSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
# 打印模型参数总数
print(f'The model has {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters')

The model has 10,316,336 trainable parameters


In [4]:
model


BidirectionalLSTM(
  (embedding): Embedding(95811, 100)
  (lstm): LSTM(100, 256, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=512, out_features=4, bias=True)
)

In [5]:
# 打印模型信息

for name, param in model.state_dict().items():
    print(name, param.size())
    print(param.dtype)

embedding.weight torch.Size([95811, 100])
torch.float32
lstm.weight_ih_l0 torch.Size([1024, 100])
torch.float32
lstm.weight_hh_l0 torch.Size([1024, 256])
torch.float32
lstm.bias_ih_l0 torch.Size([1024])
torch.float32
lstm.bias_hh_l0 torch.Size([1024])
torch.float32
lstm.weight_ih_l0_reverse torch.Size([1024, 100])
torch.float32
lstm.weight_hh_l0_reverse torch.Size([1024, 256])
torch.float32
lstm.bias_ih_l0_reverse torch.Size([1024])
torch.float32
lstm.bias_hh_l0_reverse torch.Size([1024])
torch.float32
fc.weight torch.Size([4, 512])
torch.float32
fc.bias torch.Size([4])
torch.float32


In [6]:
# 使用SGD优化器
momentum = 0.9
learning_rate = 0.1
# weight_decay = 1e-4
weight_decay = 0
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()


# 定义测试函数
def test_acc(global_model, test_loader):
    global_model.to(device)
    global_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = global_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# 训练模型
N_EPOCHS = 200
for epoch in range(N_EPOCHS):
    train_loss = 0
    model.train()
    for text, labels in dataloader_train:
        optimizer.zero_grad()
        labels = labels.to(device)
        text = text.to(device)
        predictions = model(text)
        loss = criterion(predictions, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tTest Accuracy: {test_acc(model, dataloader_test):.3f}')

    

print("Training complete")

Epoch: 01
	Train Loss: 1131.281
	Test Accuracy: 0.863
Epoch: 02
	Train Loss: 560.165
	Test Accuracy: 0.889
Epoch: 03
	Train Loss: 425.617
	Test Accuracy: 0.889
Epoch: 04
	Train Loss: 335.263
	Test Accuracy: 0.894
Epoch: 05
	Train Loss: 259.645
	Test Accuracy: 0.894
Epoch: 06
	Train Loss: 210.651
	Test Accuracy: 0.892
Epoch: 07
	Train Loss: 162.000
	Test Accuracy: 0.889
Epoch: 08
	Train Loss: 127.413
	Test Accuracy: 0.886
Epoch: 09
	Train Loss: 100.349
	Test Accuracy: 0.896
Epoch: 10
	Train Loss: 83.667
	Test Accuracy: 0.888
Epoch: 11
	Train Loss: 77.676
	Test Accuracy: 0.888
Epoch: 12
	Train Loss: 71.622
	Test Accuracy: 0.888
Epoch: 13
	Train Loss: 58.817
	Test Accuracy: 0.888
Epoch: 14
	Train Loss: 49.588
	Test Accuracy: 0.889
Epoch: 15
	Train Loss: 46.619
	Test Accuracy: 0.889
Epoch: 16
	Train Loss: 41.065
	Test Accuracy: 0.890
Epoch: 17
	Train Loss: 36.035
	Test Accuracy: 0.886
Epoch: 18
	Train Loss: 33.855
	Test Accuracy: 0.888
Epoch: 19
	Train Loss: 35.966
	Test Accuracy: 0.885
Ep

KeyboardInterrupt: 

### 使用SGD优化器
momentum = 0.9
learning_rate = 1e-3
weight_decay = 1e-4
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

Epoch: 01
	Train Loss: 2569.851
	Test Accuracy: 0.368
Epoch: 02
	Train Loss: 2494.063
	Test Accuracy: 0.394
Epoch: 03
	Train Loss: 2413.866
	Test Accuracy: 0.415
Epoch: 04
	Train Loss: 2336.511
	Test Accuracy: 0.451
Epoch: 05
	Train Loss: 2142.853
	Test Accuracy: 0.541
Epoch: 06
	Train Loss: 1644.398
	Test Accuracy: 0.702
Epoch: 07
	Train Loss: 1202.716
	Test Accuracy: 0.780
Epoch: 08
	Train Loss: 974.746
	Test Accuracy: 0.807
Epoch: 09
	Train Loss: 849.056
	Test Accuracy: 0.834
Epoch: 10
	Train Loss: 774.729
	Test Accuracy: 0.851
Epoch: 11
	Train Loss: 717.492
	Test Accuracy: 0.860
Epoch: 12
	Train Loss: 676.740
	Test Accuracy: 0.861
Epoch: 13
	Train Loss: 630.271
	Test Accuracy: 0.856
Epoch: 14
	Train Loss: 599.549
	Test Accuracy: 0.863
Epoch: 15
	Train Loss: 568.546
	Test Accuracy: 0.872
Epoch: 16
	Train Loss: 543.124
	Test Accuracy: 0.874
Epoch: 17
	Train Loss: 521.164
	Test Accuracy: 0.879
Epoch: 18
	Train Loss: 505.562
	Test Accuracy: 0.881
Epoch: 19
	Train Loss: 487.145
	Test Accuracy: 0.884
Epoch: 20
	Train Loss: 473.177
	Test Accuracy: 0.877
Epoch: 21
	Train Loss: 457.203
	Test Accuracy: 0.886
Epoch: 22
	Train Loss: 444.518
	Test Accuracy: 0.888
Epoch: 23
	Train Loss: 432.794
	Test Accuracy: 0.884
Epoch: 24
	Train Loss: 422.441
	Test Accuracy: 0.892
Epoch: 25
	Train Loss: 412.494
	Test Accuracy: 0.892
Epoch: 26
	Train Loss: 396.120
	Test Accuracy: 0.893
Epoch: 27
	Train Loss: 387.827
	Test Accuracy: 0.895
Epoch: 28
	Train Loss: 381.327
	Test Accuracy: 0.895
Epoch: 29
	Train Loss: 373.048
	Test Accuracy: 0.892
Epoch: 30
	Train Loss: 363.321
	Test Accuracy: 0.893
Epoch: 31
	Train Loss: 355.682
	Test Accuracy: 0.892
Epoch: 32
	Train Loss: 343.403
	Test Accuracy: 0.895
Epoch: 33
	Train Loss: 336.020
	Test Accuracy: 0.892
Epoch: 34
	Train Loss: 333.246
	Test Accuracy: 0.895
Epoch: 35
	Train Loss: 322.965
	Test Accuracy: 0.895
Epoch: 36
	Train Loss: 315.378
	Test Accuracy: 0.891
Epoch: 37
	Train Loss: 309.657
	Test Accuracy: 0.898
Epoch: 38
	Train Loss: 303.713
	Test Accuracy: 0.895
Epoch: 39
	Train Loss: 295.005
	Test Accuracy: 0.896
Epoch: 40
	Train Loss: 289.967
	Test Accuracy: 0.892
Epoch: 41
	Train Loss: 286.533
	Test Accuracy: 0.892
Epoch: 42
	Train Loss: 275.651
	Test Accuracy: 0.896
Epoch: 43
	Train Loss: 271.853
	Test Accuracy: 0.893
Epoch: 44
	Train Loss: 266.432
	Test Accuracy: 0.892
Epoch: 45
	Train Loss: 263.019
	Test Accuracy: 0.894
Epoch: 46
	Train Loss: 254.952
	Test Accuracy: 0.894
Epoch: 47
	Train Loss: 249.618
	Test Accuracy: 0.894
Epoch: 48
	Train Loss: 242.001
	Test Accuracy: 0.893
Epoch: 49
	Train Loss: 238.820
	Test Accuracy: 0.891
Epoch: 50
	Train Loss: 230.036
	Test Accuracy: 0.895
Epoch: 51
	Train Loss: 227.132
	Test Accuracy: 0.890
Epoch: 52
	Train Loss: 222.912
	Test Accuracy: 0.894
Epoch: 53
	Train Loss: 216.111
	Test Accuracy: 0.889
Epoch: 54
	Train Loss: 212.622
	Test Accuracy: 0.893
Epoch: 55
	Train Loss: 208.885
	Test Accuracy: 0.893
Epoch: 56
	Train Loss: 205.114
	Test Accuracy: 0.889
Epoch: 57
	Train Loss: 200.623
	Test Accuracy: 0.891
Epoch: 58
	Train Loss: 192.335
	Test Accuracy: 0.891
Epoch: 59
	Train Loss: 185.211
	Test Accuracy: 0.888
Epoch: 60
	Train Loss: 183.357
	Test Accuracy: 0.894
Epoch: 61
	Train Loss: 176.670
	Test Accuracy: 0.888
Epoch: 62
	Train Loss: 184.098
	Test Accuracy: 0.889
Epoch: 63
	Train Loss: 172.103
	Test Accuracy: 0.890
Epoch: 64
	Train Loss: 167.784
	Test Accuracy: 0.893
Epoch: 65
	Train Loss: 163.683
	Test Accuracy: 0.888
Epoch: 66
	Train Loss: 160.928
	Test Accuracy: 0.887
Epoch: 67
	Train Loss: 153.233
	Test Accuracy: 0.892
Epoch: 68
	Train Loss: 164.101
	Test Accuracy: 0.890
Epoch: 69
	Train Loss: 145.664
	Test Accuracy: 0.892
Epoch: 70
	Train Loss: 147.837
	Test Accuracy: 0.868
Epoch: 71
	Train Loss: 144.650
	Test Accuracy: 0.889
Epoch: 72
	Train Loss: 134.400
	Test Accuracy: 0.886
Epoch: 73
	Train Loss: 137.449
	Test Accuracy: 0.890
Epoch: 74
	Train Loss: 124.075
	Test Accuracy: 0.890
Epoch: 75
	Train Loss: 128.386
	Test Accuracy: 0.887
Epoch: 76
	Train Loss: 129.806
	Test Accuracy: 0.890
Epoch: 77
	Train Loss: 125.882
	Test Accuracy: 0.886
Epoch: 78
	Train Loss: 116.950
	Test Accuracy: 0.889
Epoch: 79
	Train Loss: 124.703
	Test Accuracy: 0.880
Epoch: 80
	Train Loss: 114.241
	Test Accuracy: 0.886
Epoch: 81
	Train Loss: 107.527
	Test Accuracy: 0.878
Epoch: 82
	Train Loss: 110.515
	Test Accuracy: 0.883
Epoch: 83
	Train Loss: 102.464
	Test Accuracy: 0.885
Epoch: 84
	Train Loss: 99.155
	Test Accuracy: 0.877
Epoch: 85
	Train Loss: 108.420
	Test Accuracy: 0.882
Epoch: 86
	Train Loss: 94.939
	Test Accuracy: 0.890
Epoch: 87
	Train Loss: 101.109
	Test Accuracy: 0.884
Epoch: 88
	Train Loss: 89.025
	Test Accuracy: 0.887
Epoch: 89
	Train Loss: 94.018
	Test Accuracy: 0.883
Epoch: 90
	Train Loss: 85.850
	Test Accuracy: 0.884
Epoch: 91
	Train Loss: 83.597
	Test Accuracy: 0.883
Epoch: 92
	Train Loss: 89.237
	Test Accuracy: 0.886
Epoch: 93
	Train Loss: 83.929
	Test Accuracy: 0.879
Epoch: 94
	Train Loss: 84.223
	Test Accuracy: 0.883
Epoch: 95
	Train Loss: 80.938
	Test Accuracy: 0.885
Epoch: 96
	Train Loss: 73.219
	Test Accuracy: 0.889
Epoch: 97
	Train Loss: 80.002
	Test Accuracy: 0.889
Epoch: 98
	Train Loss: 77.353
	Test Accuracy: 0.884
Epoch: 99
	Train Loss: 69.684
	Test Accuracy: 0.885
Epoch: 100
	Train Loss: 70.664
	Test Accuracy: 0.886
Epoch: 101
	Train Loss: 77.726
	Test Accuracy: 0.883
Epoch: 102
	Train Loss: 63.490
	Test Accuracy: 0.891
Epoch: 103
	Train Loss: 73.174
	Test Accuracy: 0.878
Epoch: 104
	Train Loss: 68.284
	Test Accuracy: 0.876
Epoch: 105
	Train Loss: 61.506
	Test Accuracy: 0.889
Epoch: 106
	Train Loss: 59.627
	Test Accuracy: 0.886
Epoch: 107
	Train Loss: 63.659
	Test Accuracy: 0.887
Epoch: 108
	Train Loss: 58.321
	Test Accuracy: 0.885
Epoch: 109
	Train Loss: 62.385
	Test Accuracy: 0.886
Epoch: 110
	Train Loss: 57.668
	Test Accuracy: 0.889
Epoch: 111
	Train Loss: 56.138
	Test Accuracy: 0.875
Epoch: 112
	Train Loss: 56.181
	Test Accuracy: 0.888
Epoch: 113
	Train Loss: 57.590
	Test Accuracy: 0.887
Epoch: 114
	Train Loss: 47.399
	Test Accuracy: 0.886
Epoch: 115
	Train Loss: 45.042
	Test Accuracy: 0.885
Epoch: 116
	Train Loss: 93.192
	Test Accuracy: 0.884
Epoch: 117
	Train Loss: 46.578
	Test Accuracy: 0.883
Epoch: 118
	Train Loss: 40.155
	Test Accuracy: 0.885
Epoch: 119
	Train Loss: 50.576
	Test Accuracy: 0.867
Epoch: 120
	Train Loss: 48.647
	Test Accuracy: 0.881
Epoch: 121
	Train Loss: 52.602
	Test Accuracy: 0.888
Epoch: 122
	Train Loss: 39.855
	Test Accuracy: 0.887
Epoch: 123
	Train Loss: 42.904
	Test Accuracy: 0.881
Epoch: 124
	Train Loss: 43.407
	Test Accuracy: 0.880
Epoch: 125
	Train Loss: 35.931
	Test Accuracy: 0.887
Epoch: 126
	Train Loss: 60.772
	Test Accuracy: 0.876
Epoch: 127
	Train Loss: 36.277
	Test Accuracy: 0.881
Epoch: 128
	Train Loss: 49.208
	Test Accuracy: 0.877
Epoch: 129
	Train Loss: 36.281
	Test Accuracy: 0.884
Epoch: 130
	Train Loss: 52.038
	Test Accuracy: 0.882
Epoch: 131
	Train Loss: 42.451
	Test Accuracy: 0.885
Epoch: 132
	Train Loss: 25.896
	Test Accuracy: 0.884
Epoch: 133
	Train Loss: 43.089
	Test Accuracy: 0.886
Epoch: 134
	Train Loss: 28.911
	Test Accuracy: 0.889
Epoch: 135
	Train Loss: 34.526
	Test Accuracy: 0.880
Epoch: 136
	Train Loss: 48.824
	Test Accuracy: 0.885
Epoch: 137
	Train Loss: 36.575
	Test Accuracy: 0.881
Epoch: 138
	Train Loss: 36.982
	Test Accuracy: 0.880
Epoch: 139
	Train Loss: 31.470
	Test Accuracy: 0.884
Epoch: 140
	Train Loss: 26.749
	Test Accuracy: 0.884
Epoch: 141
	Train Loss: 63.976
	Test Accuracy: 0.881
Epoch: 142
	Train Loss: 36.746
	Test Accuracy: 0.885
Epoch: 143
	Train Loss: 25.469
	Test Accuracy: 0.883
Epoch: 144
	Train Loss: 27.607
	Test Accuracy: 0.879
Epoch: 145
	Train Loss: 41.593
	Test Accuracy: 0.887
Epoch: 146
	Train Loss: 33.788
	Test Accuracy: 0.875
Epoch: 147
	Train Loss: 30.834
	Test Accuracy: 0.882
Epoch: 148
	Train Loss: 31.761
	Test Accuracy: 0.881
Epoch: 149
	Train Loss: 31.062
	Test Accuracy: 0.885
Epoch: 150
	Train Loss: 30.756
	Test Accuracy: 0.883
Epoch: 151
	Train Loss: 35.208
	Test Accuracy: 0.880
Epoch: 152
	Train Loss: 33.825
	Test Accuracy: 0.886
Epoch: 153
	Train Loss: 36.470
	Test Accuracy: 0.870
Epoch: 154
	Train Loss: 40.381
	Test Accuracy: 0.883
Epoch: 155
	Train Loss: 22.746
	Test Accuracy: 0.885
Epoch: 156
	Train Loss: 19.142
	Test Accuracy: 0.883
Epoch: 157
	Train Loss: 40.132
	Test Accuracy: 0.878
Epoch: 158
	Train Loss: 33.048
	Test Accuracy: 0.887
Epoch: 159
	Train Loss: 27.595
	Test Accuracy: 0.885
Epoch: 160
	Train Loss: 36.870
	Test Accuracy: 0.887
Epoch: 161
	Train Loss: 21.724
	Test Accuracy: 0.884
Epoch: 162
	Train Loss: 24.006
	Test Accuracy: 0.883
Epoch: 163
	Train Loss: 25.447
	Test Accuracy: 0.880
Epoch: 164
	Train Loss: 39.265
	Test Accuracy: 0.890
Epoch: 165
	Train Loss: 27.662
	Test Accuracy: 0.887
Epoch: 166
	Train Loss: 24.767
	Test Accuracy: 0.882
Epoch: 167
	Train Loss: 21.306
	Test Accuracy: 0.883
Epoch: 168
	Train Loss: 30.147
	Test Accuracy: 0.887
Epoch: 169
	Train Loss: 34.819
	Test Accuracy: 0.890
Epoch: 170
	Train Loss: 44.030
	Test Accuracy: 0.887
Epoch: 171
	Train Loss: 26.223
	Test Accuracy: 0.881
Epoch: 172
	Train Loss: 24.754
	Test Accuracy: 0.882
Epoch: 173
	Train Loss: 19.371
	Test Accuracy: 0.881
Epoch: 174
	Train Loss: 25.946
	Test Accuracy: 0.883
Epoch: 175
	Train Loss: 28.789
	Test Accuracy: 0.879
Epoch: 176
	Train Loss: 28.214
	Test Accuracy: 0.885
Epoch: 177
	Train Loss: 38.333
	Test Accuracy: 0.869
Epoch: 178
	Train Loss: 21.586
	Test Accuracy: 0.888
Epoch: 179
	Train Loss: 12.588
	Test Accuracy: 0.887
Epoch: 180
	Train Loss: 45.631
	Test Accuracy: 0.884
Epoch: 181
	Train Loss: 28.039
	Test Accuracy: 0.884
Epoch: 182
	Train Loss: 16.695
	Test Accuracy: 0.878
Epoch: 183
	Train Loss: 40.806
	Test Accuracy: 0.889
Epoch: 184
	Train Loss: 34.954
	Test Accuracy: 0.890
Epoch: 185
	Train Loss: 22.461
	Test Accuracy: 0.886
Epoch: 186
	Train Loss: 18.312
	Test Accuracy: 0.882
Epoch: 187
	Train Loss: 38.870
	Test Accuracy: 0.886
Epoch: 188
	Train Loss: 17.086
	Test Accuracy: 0.879
Epoch: 189
	Train Loss: 28.902
	Test Accuracy: 0.886
Epoch: 190
	Train Loss: 26.836
	Test Accuracy: 0.877
Epoch: 191
	Train Loss: 12.682
	Test Accuracy: 0.887
Epoch: 192
	Train Loss: 40.224
	Test Accuracy: 0.887
Epoch: 193
	Train Loss: 22.240
	Test Accuracy: 0.883
Epoch: 194
	Train Loss: 33.771
	Test Accuracy: 0.886
Epoch: 195
	Train Loss: 17.746
	Test Accuracy: 0.882
Epoch: 196
	Train Loss: 26.720
	Test Accuracy: 0.886
Epoch: 197
	Train Loss: 37.235
	Test Accuracy: 0.888
Epoch: 198
	Train Loss: 17.259
	Test Accuracy: 0.887
Epoch: 199
	Train Loss: 163.367
	Test Accuracy: 0.885
Epoch: 200
	Train Loss: 25.717
	Test Accuracy: 0.881
Training complete